In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
import copy

In [ ]:
class AntMemory:
    def __init__(self, size):
        self.memory = [0] * size
        
    def get_trash(self):
        return self.memory.count(1)
    
    def insert_value(self, value):
        self.memory.pop()
        self.memory.insert(0, value)
        return

In [ ]:
class Ant:
    def __init__(self, size):
        self.payload = 0
        self.memory = AntMemory(size)
        
    def random_move(self):
        x = y = 0
        while (x is 0 and y is 0):
            x = random.randint(-1, 1)
            y = random.randint(-1, 1)
            
        return x, y

In [ ]:
def move_ants(ants, size):
    test_ants = ants[ants['ant'].apply(lambda x: isinstance(x, Ant))]
    for _, row in test_ants.iterrows():
        ant = row['ant']
        flag = True
        while flag:
            x, y = ant.random_move()
            if (row['x'] + x) < 0 or (row['y'] + y) < 0:
                continue
            elif (row['x'] + x) == size or (row['y'] + y) == size:
                continue
            elif isinstance(ants.loc[(ants['x'] == x) & (ants['y'] == y), 'ant'], Ant):
                continue
            else:
                new_x = row['x'] + x
                new_y = row['y'] + y
                ants.loc[(ants['x'] == row['x']) & (ants['y'] == row['y'])] = [new_x, new_y, ant]
                flag = False
    return ants

In [ ]:
def calculate_pickup(ant, c_pick):
    n_trash = ant.memory.get_trash()
    size = len(ant.memory.memory)
    fx = n_trash / size
    return (c_pick / (c_pick + fx))**2

In [ ]:
def calculate_drop(ant, c_drop):
    n_trash = ant.memory.get_trash()
    size = len(ant.memory.memory)
    fx = n_trash / size
    return (fx / (c_drop + fx))**2

In [ ]:
def generate_ants_df(size, number_ants, memory_size):
    ants = pd.DataFrame(columns=['x', 'y', 'ant'])
    data = np.array([[i, j] for i in range(0, size) for j in range(0, size)])
    
    ants['x'] = data[:, 0]
    ants['y'] = data[:, 1]
    
    x_rand = random.sample(range(size), number_ants)
    y_rand = random.sample(range(size), number_ants)
    
    for i in range(len(x_rand)):
        ants.loc[(ants['x'] == x_rand[i]) & (ants['y'] == y_rand[i]), 'ant'] = Ant(memory_size)

    return ants

In [ ]:
with open('../data/test_data.pickle', 'rb') as data:
    trash = pickle.load(data)

In [ ]:
plt.figure(figsize=(20,10))
table = trash.pivot('y', 'x', 'trash')
ax = sns.heatmap(table)
ax.invert_yaxis()
plt.show()

In [ ]:
steps = 500
ants = generate_ants_df(30, 5, 5)

In [ ]:
c_pick = 0.1
c_drop = 0.3
while steps > 0:
    ants = move_ants(ants, 100)
    r = random.random()
    for index, row in ants.iterrows():
        ant = row['ant']
        if isinstance(ant, Ant):
            if ant.payload == 0:
                value = trash.loc[((trash['x'] == row['x']) & (trash['y'] == row['y'])).idxmax(), 'trash']
                if value == 1:
                    p_pick = calculate_pickup(ant, c_pick)
                    if r < p_pick:
                        ant.payload = 1
                        trash.loc[(trash['x'] == row['x']) & (trash['y'] == row['y']), 'trash'] = 0
            elif ant.payload == 1:
                value = trash.loc[((trash['x'] == row['x']) & (trash['y'] == row['y'])).idxmax(), 'trash']
                if value == 0:
                    p_drop = calculate_drop(ant, c_drop)
                    if r < p_drop:
                        ant.payload = 0
                        trash.loc[(trash['x'] == row['x']) & (trash['y'] == row['y']), 'trash'] = 1
    
    steps -= 1

In [ ]:
plt.figure(figsize=(20,10))
table = trash.pivot('y', 'x', 'trash')
ax = sns.heatmap(table)
ax.invert_yaxis()
plt.show()